# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Nicolás Martínez Gutiérrez   |
| **Fecha**      15/09/2025|   |
| **Expediente**751746 |   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

 El train_test_split se usa para comprobar si el modelo aprende de los datos y también si puede predecir bien en datos desconocidos.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

No usamos todos los datos para entrenar porque necesitamos reservar algunos para probar si el modelo realmente generaliza a nuevos casos.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [115]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [116]:
data = pd.read_excel("Motor Trend Car Road Tests.xlsx")

In [117]:
y = data["mpg"]
X = data.drop(columns=["mpg", "model"])

In [118]:
n = len(y)

# Guardar los resultados
y_pred = []
mse = []

In [119]:
for i in range(n):
    #Separar train y test
    X_train = X.drop(i) # entrenamiento = todo menos la fila i
    y_train = y.drop(i)
    X_test = X.loc[[i]] # prueba = solo la fila i
    y_test = y.loc[[i]]

    # Entrena tu modelo con las n-1 muestras.
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    # Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación
    y_true = lr.predict(X_test)
    y_pred.append(y_true)

    mse_i = mean_squared_error(y_true, y_test)
    mse.append(mse_i)

    mse_mean = np.mean(mse)
    mse_std  = np.std(mse)

In [120]:
mse

[5.258855796629032,
 2.453996713086796,
 20.550493416882723,
 0.04432910713964311,
 1.5811555159760176,
 10.118625488875402,
 0.016374041548473144,
 8.081250710389458,
 39.47018347964965,
 0.7706327722901085,
 4.95569423421588,
 10.224730198354456,
 4.430129331402713,
 0.48781277124784445,
 6.823844575809502,
 0.6027506975546979,
 36.81017860995876,
 31.759855581210513,
 1.0632275418057744,
 32.713286187027215,
 14.307173682712715,
 3.405354649998901,
 9.408007649079861,
 0.00010350039803660362,
 9.962439534547674,
 1.3412208919431314,
 0.16480064082222703,
 23.600206502249215,
 83.11457643385192,
 0.00010271237184903523,
 8.785533367966945,
 17.502929883865395]

In [121]:
mse_mean

np.float64(12.181558006901954)

In [122]:
mse_std

np.float64(17.067399871888547)

Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [123]:
from sklearn.datasets import fetch_california_housing
from sklearn.utils import shuffle

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [124]:
housing = fetch_california_housing()
X, y = shuffle(housing.data, housing.target, random_state=42)

In [125]:
k = 10
n = len(X)
fold_size = n // k
folds = np.array_split(np.arange(n), 10)

In [126]:
y_pred = []
mse = []

In [127]:
for i in range(k):
    start = i * fold_size
    end = (i + 1) * fold_size if i != k-1 else n
    
    X_test, y_test = X[start:end], y[start:end]
    X_train = np.concatenate([X[:start], X[end:]])
    y_train = np.concatenate([y[:start], y[end:]])
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_true = lr.predict(X_test)
    y_pred.append(y_true)

    mse_i = mean_squared_error(y_true, y_test)
    mse.append(mse_i)

    mse_mean = np.mean(mse)
    mse_std  = np.std(mse)


In [128]:
mse

[0.5590019245025623,
 0.5531232970331559,
 0.4811612796368581,
 0.5742036665176351,
 0.48897583781912046,
 0.5283881757036826,
 0.5511782380280955,
 0.4748396129801584,
 0.564369220719704,
 0.5500253663196308]

In [129]:
mse_mean

np.float64(0.5325266619260602)

In [130]:
mse_std

np.float64(0.03522806775489557)

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3